It looks that the **uncertainties** package is ok for funzioni linearizzabili nell'intorno di dove sono calcolate --> aka vanno bene per errori piccoli 

Per roba piú raffinata vedi:
- **soerp**: second order error propagation --> apparently is still fast
- **mcerp**: monte carlo error propagation

TODOs:
- is the emittance function linear ? (do some plots)

In [1]:
import uncertainties as un
import numpy as np
from uncertainties import unumpy

In [140]:
x = unumpy.uarray([np.pi*0.01,np.pi*0.1, np.pi*0.5, np.pi*1.],[0.02, 0.03, 0.1, 0.07])

In [141]:
def testF(xx):
    return 3*unumpy.sin(5*xx**3)

In [142]:
testF(x)

array([0.0004650941483414287+/-0.0008882643854234449,
       0.4632333191330054+/-0.13164167425082965,
       1.5149612587022716+/-9.583561164509774,
       -2.6645046553051532+/-14.286086080914606], dtype=object)

In [146]:
def theF(xx):
    return 3*np.sin(5*(xx**3))
    
def err(xx,ss):
    return (45*(xx**2)) * np.cos(5*(xx**3))*ss

In [147]:
theF(unumpy.nominal_values(x))

array([ 4.65094148e-04,  4.63233319e-01,  1.51496126e+00, -2.66450466e+00])

In [148]:
err(unumpy.nominal_values(x),unumpy.std_devs(x))

array([ 8.88264385e-04,  1.31641674e-01,  9.58356116e+00, -1.42860861e+01])

In [35]:
unumpy.nominal_values(x)

array([0.03141593, 0.31415927, 1.57079633, 3.14159265])

In [136]:
def testF(xx):
    return unumpy.sqrt(13*xx**2+xx)

def theF(xx):
    return np.sqrt(13*xx**2 + xx)
    
def err(xx,ss):
    return np.abs((26*xx +1)/(2*np.sqrt(xx*(13*xx+1))))*ss

In [137]:
testF(x)

array([0.2103483117529449+/-0.08637169819870971,
       1.2638068829930449+/-0.10881576555771832,
       5.800604333199715+/-0.3606581495068743,
       11.465009806701058+/-0.2524070509800452], dtype=object)

In [138]:
theF(unumpy.nominal_values(x))

array([ 0.21034831,  1.26380688,  5.80060433, 11.46500981])

In [139]:
err(unumpy.nominal_values(x),unumpy.std_devs(x))

array([0.0863717 , 0.10881577, 0.36065815, 0.25240705])

In [204]:
# ok mo proviamo con correlazione 
aa = unumpy.uarray([1.,2.,3.,4.,5.],[0.1, 0.1, 0.1, 0.1, 0.1])
bb = unumpy.uarray([2.,4.,6.,8.,10.],[.05, .05, .05, .05, .05])

In [205]:
import matplotlib.pyplot as plt
%matplotlib inline

In [225]:
def testF(xx,yy):
    return (xx**2+yy**2)**0.5

def theF(xx,yy):
    return np.sqrt(xx**2 + yy**2)
    
def err(xx,sx, yy, sy):
    df_x = (xx/(2*np.sqrt(xx**2+yy**2)))
    df_y = (yy/(2*np.sqrt(xx**2+yy**2)))
    return np.sqrt( df_x**2 *sx**2 + df_y**2 *sy**2)

def err_full(xx,sx, yy, sy):
    def covariance(xx,yy):
        mx = np.mean(xx)
        my = np.mean(yy)
        covar = (1./np.size(xx))*np.sum((xx-mx)*(yy-my))
        print("Linear correlation coefficient (rho) = "+str(covar/np.std(xx)/np.std(yy)))
        return covar
    
    df_x = (xx/(2*np.sqrt(xx**2+yy**2)))
    print(df_x)
    df_y = (yy/(2*np.sqrt(xx**2+yy**2)))
    return np.sqrt( (df_x*sx)**2 + (df_y*sy)**2 + (2*df_x*df_y*covariance(xx,yy)))

def err_full(xx, sx, yy, sy):
    def covariance(xx,yy):
        mx = np.mean(xx)
        my = np.mean(yy)
        covar = (1./np.size(xx))*np.sum((xx-mx)*(yy-my))
        print("Linear correlation coefficient (rho) = "+str(covar/np.std(xx)/np.std(yy)))
        return covar
    
    df_x = (xx/(2*np.sqrt(xx**2+yy**2)))
    print(df_x)
    df_y = (yy/(2*np.sqrt(xx**2+yy**2)))
    return np.sqrt( (df_x*sx)**2 + (df_y*sy)**2 + (2*df_x*df_y*covariance(xx,yy)))

In [207]:
testF(aa,bb)

array([2.23606797749979+/-0.06324555320336758,
       4.47213595499958+/-0.06324555320336758,
       6.708203932499369+/-0.0632455532033676,
       8.94427190999916+/-0.06324555320336758,
       11.180339887498949+/-0.06324555320336758], dtype=object)

In [208]:
theF(unumpy.nominal_values(aa),unumpy.nominal_values(bb))

array([ 2.23606798,  4.47213595,  6.70820393,  8.94427191, 11.18033989])

In [209]:
err(unumpy.nominal_values(aa), unumpy.std_devs(aa), unumpy.nominal_values(bb),unumpy.std_devs(bb))

array([0.03162278, 0.03162278, 0.03162278, 0.03162278, 0.03162278])

In [210]:
err_full(unumpy.nominal_values(aa), unumpy.std_devs(aa), unumpy.nominal_values(bb),unumpy.std_devs(bb))

[0.2236068 0.2236068 0.2236068 0.2236068 0.2236068]
Linear correlation coefficient (rho) = 0.9999999999999999


array([0.89498603, 0.89498603, 0.89498603, 0.89498603, 0.89498603])

In [214]:
np.cov(unumpy.nominal_values(aa),unumpy.nominal_values(bb))

array([[ 2.5,  5. ],
       [ 5. , 10. ]])

In [215]:
2*0.2236068*0.2236068*5

0.5000000100624

In [224]:
np.cov(unumpy.nominal_values(aa),unumpy.nominal_values(bb), ddof=1)

array([[ 2.5,  5. ],
       [ 5. , 10. ]])